# Analyzing Market Elasticity of Housing Prices 

Data sourced from Professor Christopher Brooks at the University of Michigan  
*Data files used: [City_Zhvi_AllHomes.csv](https://www.zillow.com/research/data/), [university_towns.txt](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States), [gdplev.xls](https://www.bea.gov/data/gdp/gross-domestic-product#gdp)*  
  
This project focuses on aggregating data from various sources stored in different formats to showcase statistical analysis capabilities, specifically hypothesis testing.  The project develops a research question beforehand to guide our process.  The data used pertains to quarterly periods between the years 2000 and 2016.  The project employs various techniques using methods found in the Numpy, Pandas, and SciPy libraries. 

## Research Question

How insulated are housing prices in US University towns from cyclical movements in the economy?  Are housing prices in University towns more resilient during recessions than non-university towns?

### Hypothesis Question

Did University towns have their mean housing prices affected differently by the late 2000's than towns without a university?

### Method

I will be conducting a two-sided, difference of means t-test in comparing prices between towns with and without universities.  The metric I will be using will be comparing the mean Price Ratio of each group as defined by:  

*Price Ratio = GDP Quarter before recession / GDP middle of the recession*

Null Hypothesis: PR(uni) = PR(non_uni)

### Step 1: Load and clean data

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import scipy.stats as stats 

In [2]:
# map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Remove newline character '\n'. '''
        
    towns = [line.rstrip() for line in open('university_towns.txt')]
    lst = []
    state=''
    region=''
    for site in towns:
        if '[ed' in site:
            s = site.split('[ed')
            state = s[0]
        else:
            r = site.split(' (')
            region = r[0]
        lst.append([state, region])
        region = ''
    for s in lst:
        if s[1] == '':
            lst.remove(s)
    df = pd.DataFrame(lst, columns= ['State', 'RegionName'])
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


### Step 2: Determine start, middle, and end of recession period

In [4]:
def get_recession_start():
    gdp_raw = pd.read_excel('gdplev.xls', skiprows = 7) # cleaning data 
    qgdp = gdp_raw.loc[:,'Unnamed: 4': 'Unnamed: 6'] # rename columns
    gdp = qgdp.drop('Unnamed: 5', axis = 1)
    gdp.rename(columns = {'Unnamed: 4': 'Q GDP', 'Unnamed: 6': '$Bn GDP'}, inplace = True)
    start = gdp[gdp['Q GDP'] == '2000q1'].index[0]
    gdp = gdp[start:]
    gdp = gdp.set_index('Q GDP')
    dollars = np.array(gdp['$Bn GDP'])
    movement = []
    last = dollars[0]
    for q in dollars: # label direction of change from the previous quarter
        d = q - last
        if d > 0:
            movement.append(1)
        elif d < 0:
            movement.append(-1)
        else:
            movement.append(0)
        last = q
    gdp['Change'] = movement
    start_date = []
    date_count = 1
    for date in gdp['Change'][1:]: # recessions are defined as two consecutive quarters of GDP contraction
        if date == -1 and gdp['Change'][date_count - 1] == -1 and gdp['Change'][date_count - 2] != -1:
            start_date.append(gdp['Change'].index[date_count - 1])
        date_count += 1
    return start_date[0]

In [5]:
def get_recession_end():
    gdp_raw = pd.read_excel('gdplev.xls', skiprows = 7)
    qgdp = gdp_raw.loc[:,'Unnamed: 4': 'Unnamed: 6'] # same cleaning as function prior
    gdp = qgdp.drop('Unnamed: 5', axis = 1) 
    gdp.rename(columns = {'Unnamed: 4': 'Q GDP', 'Unnamed: 6': '$Bn GDP'}, inplace = True)
    start = gdp[gdp['Q GDP'] == '2000q1'].index[0]
    gdp = gdp[start:]
    gdp = gdp.set_index('Q GDP')
    dollars = np.array(gdp['$Bn GDP'])
    movement = []
    last = dollars[0]
    for q in dollars:
        d = q - last
        if d > 0:
            movement.append(1)
        elif d < 0:
            movement.append(-1)
        else:
            movement.append(0)
        last = q
    gdp['Change'] = movement
    start_date = []
    end_date = []
    date_count = 1 # acts as a count to find two consecutive quarters of GDP growth after recession starts
    d_count = 0
    for date in gdp['Change'][1:]: # first month
        if date == -1 and gdp['Change'][date_count - 1] == -1 and gdp['Change'][date_count - 2] != -1:
            start_date.append(gdp['Change'].index[date_count - 1]) 
            d_count = date_count
            for d in gdp['Change'][date_count:]: # last month
                if d == 1 and gdp['Change'][d_count - 1] == 1 and gdp['Change'][d_count - 2] != 1 and gdp['Change'][d_count - 3] != 1:
                    end_date.append(gdp['Change'].index[d_count])
                d_count += 1
        date_count += 1
    return end_date[0]

In [6]:
def get_recession_bottom():
    gdp_raw = pd.read_excel('gdplev.xls', skiprows = 7)
    qgdp = gdp_raw.loc[:,'Unnamed: 4': 'Unnamed: 6']
    gdp = qgdp.drop('Unnamed: 5', axis = 1)
    gdp.rename(columns = {'Unnamed: 4': 'Q GDP', 'Unnamed: 6': '$Bn GDP'}, inplace = True) # renmaed unnamed columns
    start = gdp[gdp['Q GDP'] == '2000q1'].index[0]
    gdp = gdp[start:]
    gdp = gdp.set_index('Q GDP')
    dollars = np.array(gdp['$Bn GDP'])
    first = get_recession_start()
    last = get_recession_end()
    rec = gdp.loc[first:last,:]
    bottom = rec[rec['$Bn GDP'] == rec['$Bn GDP'].min()] # finds worst quarter of the recession
    b_q = bottom['$Bn GDP'].index[0]
    return b_q

### Step 3: Convert monthly housing data in quarters to match GDP dataset

In [11]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    hdf = pd.read_csv('City_Zhvi_AllHomes.csv')
    l = list(hdf.columns)
    first_cut = l.index('1996-04')
    last_cut = l.index('1999-12') + 1 
    to_drop_prices = hdf.columns[first_cut:last_cut]
    h = hdf.drop(to_drop_prices, axis=1)
    h = h.set_index(['State', 'RegionName'])
    h = h.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis = 1)
    h_q = h.groupby(np.arange(len(h.columns))//3, axis=1).mean() # takes mean of the months grouped by each quarter
    new_dates = []
    for year in range(2000,2017):
        for q in range(1,5): # formalize coulumn labels
            date = '{}q{}'.format(year, q)
            new_dates.append(date)
    dates_for_table = new_dates[:-1]
    h_q.columns = dates_for_table
    h_q = h_q.rename(states)
    return h_q

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


### Step 4: Build and run T-test

In [12]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothethis is true or not as well as the p-value of the confidence. 
    
    Returns the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    r_start = get_recession_start() # load in data
    r_bottom = get_recession_bottom()
    t_data = convert_housing_data_to_quarters()
    p = list(t_data.columns)
    start = p[p.index(r_start) -1]
    t = t_data[[start, r_bottom]]
    t['Price Ratio'] = t.apply(lambda x: (x['2008q2']) / x['2009q2'], axis=1) # new column for ratio
    u = get_list_of_university_towns()
    u = u.reset_index()
    t = t.reset_index()
    unni = pd.merge(t, u, how='inner', left_on=['State','RegionName'], right_on=['State','RegionName']) # create two samples, inner join here to match college towns
    no_unni = pd.merge(t, u, how = 'outer', left_on=['State','RegionName'], right_on=['State','RegionName']) # outer join to match towns that are not on university towns list
    results = stats.ttest_ind(unni['Price Ratio'], no_unni['Price Ratio'], nan_policy='omit') # do not count NaNs
    p = results[1]
    if p <0.01:
        different = True
    else:
        different = False
    if unni['Price Ratio'].mean() < no_unni['Price Ratio'].mean():
        better = "university town"
    else:
        better = "non-university town"
    return different, p, better

run_ttest()

/Users/Ryan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(True, 0.0033861978483578884, 'university town')

## Results

The two-sampled, two-tailed t-test found there be statistically significant evidnece to suggest that mean prices for homes in university towns is different than homes in non-university towns.  Moreover, the difference suggests that prices in university towns decline at a reduced rate to other towns.  Given the P-value to be 0.003, it can be said that this difference is highly significant at an alpha level of 0.005, the threshold to reject the null of a one-sided difference of means test (given alpha = 0.01).

We can therefore conclude that housing prices in college towns decline at a less drastic rate than non-college towns are are more insulated to market downturns and economic recessions.